In [ ]:
# import modules
import pandas as pd
import numpy as np
import os

# Work on Crime and Clearances, and Pop and Area

In [236]:
# DataFrame for Crimes and clearances with Arson
crime_clearance_df = pd.read_csv(
    "../data/Crimes_and_Clearances_with_Arson-1985-2023.csv"
)

# DataFrame for Pop and area by county
pop_area = pd.read_csv("../data/Pop_and_area_by_county_1980_to_2024.csv")

C:\Users\leoaf\AppData\Local\Temp\ipykernel_148\985258103.py:2: DtypeWarning: Columns (21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_clearance_df = pd.read_csv(


The message above indicates that some of the columns have mixed data type. We can resolve this below:
- First check which columns have non-numeric data types.
- Check which columns have mixed data types. 

In [237]:
## Columns with Non numeric dtypes
non_numeric_cols = crime_clearance_df.select_dtypes(include=["object"]).columns
print(non_numeric_cols)

Index(['County', 'NCICCode', 'TotalStructural_sum', 'TotalMobile_sum',
       'TotalOther_sum', 'GrandTotal_sum', 'GrandTotClr_sum'],
      dtype='object')


In [238]:
def mixed_type_columns(df: pd.DataFrame) -> list:
    """
    Look for columns with mixed types

    Parameters:
    df(pd.DataFrame)

    Returns:
    list: A list of columes with mixed dtypes
    """
    mixed_columns = []
    for column in df.columns:
        types_in_column = df[column].map(type).unique()
        if len(types_in_column) > 1:
            mixed_columns.append(column)
    return mixed_columns

In [239]:
# The columns with mixed dtypes
mixed_columns = mixed_type_columns(crime_clearance_df)
print(mixed_columns)

['TotalStructural_sum', 'TotalMobile_sum', 'TotalOther_sum', 'GrandTotal_sum', 'GrandTotClr_sum']


In [240]:
cca_df = crime_clearance_df.copy()  # A copy of rime_clearance_df

# Resolve the issue with mixed dtypes
cca_df[mixed_columns] = cca_df[mixed_columns].apply(pd.to_numeric, errors="coerce")

In [241]:
# No mixed types in the copy of the dataframe.
mixed_columns = mixed_type_columns(cca_df)
print(mixed_columns == [])

True


In [242]:
# Remove the column
cca_df = cca_df.drop(["NCICCode"], axis=1)
cca_df.head()

,Year,County,Violent_sum,Homicide_sum,ForRape_sum,Robbery_sum,AggAssault_sum,Property_sum,Burglary_sum,VehicleTheft_sum,...,MVPLARnao_sum,BILARnao_sum,FBLARnao_sum,COMLARnao_sum,AOLARnao_sum,LT400nao_sum,LT200400nao_sum,LT200nao_sum,LT50200nao_sum,LT50nao_sum
0,1985,Alameda County,427,3,27,166,231,3964,1483,353,...,109,205,44,11,475,753.0,437.0,NaN,440,498
1,1985,Alameda County,405,7,15,220,163,4486,989,260,...,673,516,183,53,559,540.0,622.0,NaN,916,1159
2,1985,Alameda County,101,1,4,58,38,634,161,55,...,62,39,46,17,37,84.0,68.0,NaN,128,138
3,1985,Alameda County,1164,11,43,660,450,12035,2930,869,...,508,611,1877,18,496,533.0,636.0,NaN,2793,4274
4,1985,Alameda County,146,0,5,82,59,971,205,102,...,153,16,85,24,169,217.0,122.0,NaN,161,164


In [243]:
# Custom function to remove County from the values in the column County


def remove_county(text: str) -> str:
    """
    Remove County from string
    """
    return text.replace(" County", "")


assert remove_county("Hello County") == "Hello"
assert remove_county("Hello World County") == "Hello World"

In [244]:
# Apply the remove_county to the dataframe cca_df
cca_df["County"] = cca_df["County"].apply(remove_county)
cca_df["Year"] = cca_df["Year"].astype(str)

# Group by 'County' and 'Year'
cca_grouped_df = cca_df.groupby(["County", "Year"]).sum()

In [245]:
cca_grouped_df.head(3)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   

              AggAssault_sum  Property_sum  Burglary_sum  VehicleTheft_sum  \
County  Year                                                                 
Alameda 1985            5267         89297         24997              7142   
        1986            5530         90167         24392              7896   
        1987            5767         88306         22399              8909   

              LTtotal_sum  ViolentClr_sum  ...  MVPLARnao_sum  BILARnao_sum  \
County  Year                               ...                                
Alameda 1985        57158            5429  ...           5728          3926   
        1986        57879            5570  ...           5449          3380   
        1987        56998            6303  ...           5445          2954   

              FBLARnao_sum  COMLARnao_sum  AOLARnao_sum  LT400nao_sum  \
County  Year                                                            
Alameda 1985         10664            406          9163        7744.0   
        1986          9575            465          9552        9048.0   
        1987          8687            256          8817       11437.0   

              LT200400nao_sum  LT200nao_sum  LT50200nao_sum  LT50nao_sum  
County  Year                                                              
Alameda 1985           7787.0           0.0           14473        27154  
        1986           7482.0           0.0           13459        27890  
        1987           8132.0           0.0           10845        26584  

[3 rows x 67 columns]

In [246]:
# Create a new feature 'crime_rate' for each county and year: crime_rate = Violentsum/poplation
violent = [
    "Violent_sum",
    "Homicide_sum",
    "ForRape_sum",
    "Robbery_sum",
    "AggAssault_sum",
    "ViolentClr_sum",
    "HomicideClr_sum",
    "ForRapeClr_sum",
    "RobberyClr_sum",
    "AggAssaultClr_sum",
]

property = [
    "Property_sum",
    "Burglary_sum",
    "VehicleTheft_sum",
    "LTtotal_sum",
    "PropertyClr_sum",
    "BurglaryClr_sum",
    "VehicleTheftClr_sum",
    "LTtotalClr_sum",
]

In [247]:
crime_data = cca_grouped_df[violent + property]
crime_data.head()

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   
        1988        10963           159          722         4863   
        1989        10563           172          670         4879   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   
        1988            5219            5708              100             498   
        1989            4842            5250               98             453   

              RobberyClr_sum  AggAssaultClr_sum  Property_sum  Burglary_sum  \
County  Year                                                                  
Alameda 1985            1517               3376         89297         24997   
        1986            1545               3379         90167         24392   
        1987            1569               4132         88306         22399   
        1988            1545               3565         92745         22308   
        1989            1496               3203         92888         21311   

              VehicleTheft_sum  LTtotal_sum  PropertyClr_sum  BurglaryClr_sum  \
County  Year                                                                    
Alameda 1985              7142        57158            15409             3117   
        1986              7896        57879            15121             2899   
        1987              8909        56998            16380             2848   
        1988             11080        59357            16747             2671   
        1989             12556        59021            16171             2539   

              VehicleTheftClr_sum  LTtotalClr_sum  
County  Year                                       
Alameda 1985                 1607           10685  
        1986                 1698           10524  
        1987                 2189           11343  
        1988                 2533           11543  
        1989                 2560           11072

In [248]:
# Columns for new dataframe
list_of_cols = [
    "year",
    "county",
    "population",
    "crime_rate",
    "clearance_rate",
    "population_density",
    "vacancy_rate",
    "number_of_person_in_household",
    "mobile_home_ratio",
    "percent_in_poverty",
    "adjusted_median_income",
    "unemployment_rate",
    "dropout_rate",
    "public_school_rate",
    "no_highschool_rate",
    "uninsured_rate",
    "house_affordability",
    "adj_police_budget",
    "adj_education_budget",
    "adj_welfare_budget",
    "adj_mental_health_budget",
    "adj_rehab_budget",
    "adj_health_budget",
    "adj_judiciary_budget",
    "adj_prison_budget",
    "median_age",
    "home_ownership_rate",
    "rent_burden",
]

In [249]:
pop_area = pop_area.rename(columns={"COUNTY": "County", "Area (sq mi)": "Area_sq_mi"})
pop_area = pop_area.fillna(0)

pop_area_copy = pop_area.copy()  # Make a copy
pop_area_copy.head(3)

,County,Area_sq_mi,1981,1982,1983,1984,1985,1986,1987,1988,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Alameda,738.0,"1,117,800","1,134,000","1,151,800","1,170,400","1,185,500","1,206,900","1,220,600","1,242,300",...,"1,622,205","1,641,983","1,656,919","1,666,247","1,675,964","1,681,337","1,655,767","1,645,265","1,644,199","1,644,569"
1,Alpine,739.0,"1,090","1,100","1,120","1,080","1,100","1,140","1,130","1,100",...,"1,190","1,196","1,201","1,205","1,201","1,204","1,181","1,177","1,166","1,163"
2,Amador,606.0,"19,800","20,250","20,600","21,050","21,800","22,450","23,300","25,750",...,"37,453","37,663","38,807","39,708","40,227","40,426","40,224","40,073","40,028","39,893"


In [250]:
pop_area_copy = pop_area_copy.drop(["Area_sq_mi"], axis=1)
pop_area_copy["County"] = pop_area_copy["County"].apply(lambda x: str(x.strip()))
pop_area_copy.head()

,County,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Alameda,"1,117,800","1,134,000","1,151,800","1,170,400","1,185,500","1,206,900","1,220,600","1,242,300","1,261,200",...,"1,622,205","1,641,983","1,656,919","1,666,247","1,675,964","1,681,337","1,655,767","1,645,265","1,644,199","1,644,569"
1,Alpine,"1,090","1,100","1,120","1,080","1,100","1,140","1,130","1,100","1,090",...,"1,190","1,196","1,201","1,205","1,201","1,204","1,181","1,177","1,166","1,163"
2,Amador,"19,800","20,250","20,600","21,050","21,800","22,450","23,300","25,750","27,600",...,"37,453","37,663","38,807","39,708","40,227","40,426","40,224","40,073","40,028","39,893"
3,Butte,"146,800","150,700","153,800","156,600","159,700","163,000","166,200","170,800","175,200",...,"227,400","228,198","230,412","231,774","227,263","216,090","206,058","206,183","205,741","206,194"
4,Calaveras,"21,350","22,250","23,200","23,850","24,650","25,550","26,800","28,200","29,700",...,"45,395","45,402","45,355","45,367","45,324","45,290","45,013","44,771","44,616","44,436"


In [251]:
# Convert pop_area_copy to the structure of crime_data
pop_index_county = pop_area_copy.set_index("County")  # Index dataframe by County
pop_stacked = pop_index_county.stack().to_frame(name="popupation")
pop_stacked["popupation"] = pop_stacked["popupation"].apply(
    lambda x: int(str(x.replace(",", "")))
)
pop_stacked.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2596 entries, ('Alameda', '1981') to ('State Total', '2024')
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   popupation  2596 non-null   int64
dtypes: int64(1)
memory usage: 29.3+ KB


In [252]:
# Create Column for Population and Area

crime_data_df = crime_data.copy()
crime_data_df["Population"] = pop_stacked[
    "popupation"
]  # Add column 'Population' to crime_data_df

area_df = pop_area.copy()
area_df = area_df[["County", "Area_sq_mi"]]
area_df["County"] = area_df["County"].apply(lambda x: str(x.strip()))

crime_data_df = crime_data_df.reset_index()  # Reset the index
crime_data_df = crime_data_df.merge(area_df, on="County", how="left")
crime_data_df = crime_data_df.set_index(["County", "Year"])

In [253]:
crime_data_df.head(3)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   

              RobberyClr_sum  AggAssaultClr_sum  Property_sum  Burglary_sum  \
County  Year                                                                  
Alameda 1985            1517               3376         89297         24997   
        1986            1545               3379         90167         24392   
        1987            1569               4132         88306         22399   

              VehicleTheft_sum  LTtotal_sum  PropertyClr_sum  BurglaryClr_sum  \
County  Year                                                                    
Alameda 1985              7142        57158            15409             3117   
        1986              7896        57879            15121             2899   
        1987              8909        56998            16380             2848   

              VehicleTheftClr_sum  LTtotalClr_sum  Population  Area_sq_mi  
County  Year                                                               
Alameda 1985                 1607           10685     1185500       738.0  
        1986                 1698           10524     1206900       738.0  
        1987                 2189           11343     1220600       738.0

## Some features
- Crime rate
- Claerance rate
- Population density

In [254]:
crime_data_df["crime_rate"] = (
    crime_data_df["Violent_sum"] / crime_data_df["Population"]
)  # Crime rate
crime_data_df["clearance_rate"] = (
    crime_data_df["ViolentClr_sum"] / crime_data_df["Population"]
)  # Clearance rate
crime_data_df["population_density"] = (
    crime_data_df["Population"] / crime_data_df["Area_sq_mi"]
)

In [255]:
crime_data_df.head(3)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   

              RobberyClr_sum  AggAssaultClr_sum  ...  LTtotal_sum  \
County  Year                                     ...                
Alameda 1985            1517               3376  ...        57158   
        1986            1545               3379  ...        57879   
        1987            1569               4132  ...        56998   

              PropertyClr_sum  BurglaryClr_sum  VehicleTheftClr_sum  \
County  Year                                                          
Alameda 1985            15409             3117                 1607   
        1986            15121             2899                 1698   
        1987            16380             2848                 2189   

              LTtotalClr_sum  Population  Area_sq_mi  crime_rate  \
County  Year                                                       
Alameda 1985           10685     1185500       738.0    0.009809   
        1986           10524     1206900       738.0    0.010353   
        1987           11343     1220600       738.0    0.009588   

              clearance_rate  population_density  
County  Year                                      
Alameda 1985        0.004580         1606.368564  
        1986        0.004615         1635.365854  
        1987        0.005164         1653.929539  

[3 rows x 23 columns]

# Work on Unemployment 

In [256]:
# DataFrame for Unemployment rate
unemployment_rate_df = pd.read_excel("../data/Unemployment_rate_1990-2023.xlsx")

In [257]:
unemployment_rate_df = unemployment_rate_df.rename(
    columns={
        "County Name/State Abbreviation": "County",
        "unemployment rate(%)": "unemployment_rate",
    }
)
unemployment_rate_df.head()

,Code,Code.1,Code.2,County,Year,Unnamed: 5,Laber Force,Employed,Unemployed,unemployment_rate
0,CN0600100000000,6,1,"Alameda County, CA",2023,NaN,826102,792439,33663,4.1
1,CN0600300000000,6,3,"Alpine County, CA",2023,NaN,540,505,35,6.5
2,CN0600500000000,6,5,"Amador County, CA",2023,NaN,14404,13673,731,5.1
3,CN0600700000000,6,7,"Butte County, CA",2023,NaN,91910,87088,4822,5.2
4,CN0600900000000,6,9,"Calaveras County, CA",2023,NaN,21956,21030,926,4.2


In [258]:
unemployment_rate_df["County"] = unemployment_rate_df["County"].apply(
    lambda x: x.replace(" County, CA", "").replace(" County/city, CA", "")
)
unemp_rate_df = unemployment_rate_df.copy()
unemp_rate_df = unemp_rate_df.fillna(0)
unemp_rate_df = unemp_rate_df[["County", "Year", "unemployment_rate"]]
unemp_rate_df["Year"] = unemp_rate_df["Year"].astype(str)
unemp_rate_df.set_index(["County", "Year"], inplace=True)

In [259]:
crime_data_df = crime_data_df.merge(unemp_rate_df, on=["County", "Year"], how="left")

In [260]:
crime_data_df.head(5)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   
        1988        10963           159          722         4863   
        1989        10563           172          670         4879   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   
        1988            5219            5708              100             498   
        1989            4842            5250               98             453   

              RobberyClr_sum  AggAssaultClr_sum  ...  PropertyClr_sum  \
County  Year                                     ...                    
Alameda 1985            1517               3376  ...            15409   
        1986            1545               3379  ...            15121   
        1987            1569               4132  ...            16380   
        1988            1545               3565  ...            16747   
        1989            1496               3203  ...            16171   

              BurglaryClr_sum  VehicleTheftClr_sum  LTtotalClr_sum  \
County  Year                                                         
Alameda 1985             3117                 1607           10685   
        1986             2899                 1698           10524   
        1987             2848                 2189           11343   
        1988             2671                 2533           11543   
        1989             2539                 2560           11072   

              Population  Area_sq_mi  crime_rate  clearance_rate  \
County  Year                                                       
Alameda 1985     1185500       738.0    0.009809        0.004580   
        1986     1206900       738.0    0.010353        0.004615   
        1987     1220600       738.0    0.009588        0.005164   
        1988     1242300       738.0    0.008825        0.004595   
        1989     1261200       738.0    0.008375        0.004163   

              population_density  unemployment_rate  
County  Year                                         
Alameda 1985         1606.368564                NaN  
        1986         1635.365854                NaN  
        1987         1653.929539                NaN  
        1988         1683.333333                NaN  
        1989         1708.943089                NaN  

[5 rows x 24 columns]

# Work on Median House and CPI

In [261]:
# DataFrame for Califonia_CPI
califonia_cpi_df = pd.read_excel("../data/California_CPI_1985_to_2023.xlsx")

# DataFrame for median household income
median_house_income = pd.read_excel("../data/Median_income_2000_and_2009_to_2023.xlsx")

In [262]:
califonia_cpi_df["Year"] = califonia_cpi_df["Year"].astype(str)

In [263]:
median_house_income["Year"] = median_house_income["Year"].astype(str)

In [264]:
median_house_cpi = median_house_income.merge(califonia_cpi_df, on="Year", how="left")
median_house_cpi["County"] = median_house_cpi["County"].apply(
    lambda x: x.replace(" County", "").strip()
)
median_house_cpi = median_house_cpi.rename(
    columns={"Median Household Income": "median_hse_income"}
)
median_house_cpi = median_house_cpi.set_index(["County", "Year"])
median_house_cpi["median_hse_income"] = median_house_cpi["median_hse_income"].astype(
    "float64"
)

In [265]:
median_house_cpi["adjusted_income"] = (
    median_house_cpi["median_hse_income"] / median_house_cpi.CPI
)
median_house_cpi.head(3)

,,median_hse_income,CPI,adjusted_income
County,Year,,,
California,2023,95473.0,331.804,287.739147
Alameda,2023,119230.0,331.804,359.338646
Alpine,2023,83265.0,331.804,250.946342


In [266]:
crime_data_df = crime_data_df.merge(median_house_cpi, on=["County", "Year"], how="left")

In [267]:
crime_data_df.head(5)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   
        1988        10963           159          722         4863   
        1989        10563           172          670         4879   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   
        1988            5219            5708              100             498   
        1989            4842            5250               98             453   

              RobberyClr_sum  AggAssaultClr_sum  ...  LTtotalClr_sum  \
County  Year                                     ...                   
Alameda 1985            1517               3376  ...           10685   
        1986            1545               3379  ...           10524   
        1987            1569               4132  ...           11343   
        1988            1545               3565  ...           11543   
        1989            1496               3203  ...           11072   

              Population  Area_sq_mi  crime_rate  clearance_rate  \
County  Year                                                       
Alameda 1985     1185500       738.0    0.009809        0.004580   
        1986     1206900       738.0    0.010353        0.004615   
        1987     1220600       738.0    0.009588        0.005164   
        1988     1242300       738.0    0.008825        0.004595   
        1989     1261200       738.0    0.008375        0.004163   

              population_density  unemployment_rate  median_hse_income  CPI  \
County  Year                                                                  
Alameda 1985         1606.368564                NaN                NaN  NaN   
        1986         1635.365854                NaN                NaN  NaN   
        1987         1653.929539                NaN                NaN  NaN   
        1988         1683.333333                NaN                NaN  NaN   
        1989         1708.943089                NaN                NaN  NaN   

              adjusted_income  
County  Year                   
Alameda 1985              NaN  
        1986              NaN  
        1987              NaN  
        1988              NaN  
        1989              NaN  

[5 rows x 27 columns]

# Work on Poverty

In [268]:
poverty_rate_df = pd.read_excel("../data/Poverty_rate_2009_2023.xlsx")

In [269]:
poverty_rate_df.head()

,Year,ID,Name,Poverty Universe,Number in Poverty,90% Confidence Interval,Percent in Poverty,90% Confidence Interval.1
0,2023,6000,California,38249913,4597732,"4,546,196 to 4,649,268",12.0,11.9 to 12.1
1,2023,6001,Alameda County,1594026,151872,"138,959 to 164,785",9.5,8.7 to 10.3
2,2023,6003,Alpine County,1136,177,134 to 220,15.6,11.8 to 19.4
3,2023,6005,Amador County,37700,4400,"3,493 to 5,307",11.7,9.3 to 14.1
4,2023,6007,Butte County,203267,40532,"36,792 to 44,272",19.9,18.1 to 21.7


# Work on health Insurance

In [270]:
health_insurance_df = pd.read_excel("../data/Health_Insurance_2010_to_2023.xlsx") # Single file

In [ ]:
# Combine all files in Extra-HI

extra_hi_dir = "../data/Extra-HI"

hi_dfs = {} # Dictionary to store data frames

# Loop over a list of files in the directory: Extra-HI
for filename in os.listdir(extra_hi_dir):
    # print(filename)
    file_path = os.path.join(extra_hi_dir, filename)
    raw_data = pd.read_excel(file_path, sheet_name="Data", header=[0,1]) 
    year = filename.strip("HI_, .xlsx")
    trans = raw_data.T
    trans["Year"] = year
    hi_dfs[year] = trans

### Combine all the dataframes into one dataframe 

In [ ]:
#Combine all the dataframes into one dataframe

combine_his = pd.concat(hi_dfs.values(), axis=0) 
combine_his.columns= combine_his.iloc[0]
combine_his=combine_his[1:]
combine_his = combine_his.reset_index()
combine_his=combine_his.rename(columns={'2010':'Year', 'level_0':'County', 'level_1':'Label'}) 
combine_his["County"] = combine_his["County"].apply(
    lambda x: x.replace(" County, California", "").strip()
)
combine_his = combine_his.set_index(['County', 'Year'])


In [273]:
combine_his.head(3)

('Unnamed: 0_level_0', 'Label')                     Label EMPLOYMENT STATUS  \
County                     Year                                               
Alameda County, California 2010                  Estimate               NaN   
                           2010  Estimate Margin of Error               NaN   
                           2010                   Percent               NaN   

('Unnamed: 0_level_0', 'Label') Population 16 years and over In labor force  \
County                     Year                                               
Alameda County, California 2010                    1,211,230        797,135   
                           2010                       ±1,833         ±9,451   
                           2010                    1,211,230          65.8%   

('Unnamed: 0_level_0', 'Label') Civilian labor force Employed Unemployed  \
County                     Year                                            
Alameda County, California 2010              796,058  699,389     96,669   
                           2010               ±9,453   ±9,766     ±5,653   
                           2010                65.7%    57.7%       8.0%   

('Unnamed: 0_level_0', 'Label') Armed Forces Not in labor force  \
County                     Year                                   
Alameda County, California 2010        1,077            414,095   
                           2010         ±458             ±9,725   
                           2010         0.1%              34.2%   

('Unnamed: 0_level_0', 'Label') Civilian labor force  ... All people  \
County                     Year                       ...              
Alameda County, California 2010              796,058  ...        (X)   
                           2010               ±9,453  ...        (X)   
                           2010              796,058  ...      13.5%   

('Unnamed: 0_level_0', 'Label') Under 18 years  \
County                     Year                  
Alameda County, California 2010            (X)   
                           2010            (X)   
                           2010          17.4%   

('Unnamed: 0_level_0', 'Label') Related children under 18 years  \
County                     Year                                   
Alameda County, California 2010                             (X)   
                           2010                             (X)   
                           2010                           17.2%   

('Unnamed: 0_level_0', 'Label') Related children under 5 years  \
County                     Year                                  
Alameda County, California 2010                            (X)   
                           2010                            (X)   
                           2010                          15.9%   

('Unnamed: 0_level_0', 'Label') Related children 5 to 17 years  \
County                     Year                                  
Alameda County, California 2010                            (X)   
                           2010                            (X)   
                           2010                          17.6%   

('Unnamed: 0_level_0', 'Label') 18 years and over 18 to 64 years  \
County                     Year                                    
Alameda County, California 2010               (X)            (X)   
                           2010               (X)            (X)   
                           2010             12.3%          12.7%   

('Unnamed: 0_level_0', 'Label') 65 years and over People in families  \
County                     Year                                        
Alameda County, California 2010               (X)                (X)   
                           2010               (X)                (X)   
                           2010             10.1%              10.6%   

('Unnamed: 0_level_0', 'Label') Unrelated individuals 15 years and over  
County                     Year                                          
Alameda County, California 2010

In [274]:
health_insurance_df.head()

,Year,County,Label,Civilian noninstitutionalized population,With health insurance coverage,With private health insurance,With public coverage,No health insurance coverage
0,2023,"Alameda County, California",Estimate,1641321,1574283,1228986,502254,67038
1,2023,"Alpine County, California",Estimate,1695,1610,1119,734,85
2,2023,"Amador County, California",Estimate,37789,35568,26408,17708,2221
3,2023,"Butte County, California",Estimate,207385,194212,127086,98182,13173
4,2023,"Calaveras County, California",Estimate,45670,43263,28952,22699,2407


In [275]:
health_insurance_df.tail()

,Year,County,Label,Civilian noninstitutionalized population,With health insurance coverage,With private health insurance,With public coverage,No health insurance coverage
1547,2010,"Sutter County, California",Percent,"92,664",79.3%,51.1%,38.3%,20.7%
1548,2010,"Tulare County, California",Percent,"440,539",78.1%,46.6%,39.4%,21.9%
1549,2010,"Ventura County, California",Percent,"816,034",83.7%,68.3%,25.0%,16.3%
1550,2010,"Yolo County, California",Percent,"199,916",88.2%,73.7%,23.0%,11.8%
1551,2010,"Yuba County, California",Percent,"68,357",83.2%,49.4%,44.7%,16.8%


# Work on House Ownership data

In [276]:
hse_ownership_df = pd.read_excel(
    "../data/House_Ownership_rent_2010_to_2023.xlsx", header=None
)

In [277]:
hse_ownership_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248
0,Year,2023,2023,2023,2023,2023,2023,2023,2023,2023,...,2010,2010,2010,2010,2010,2010,2010,2010,2010,2010
1,County,"Alameda County, California","Alameda County, California","Alameda County, California","Alameda County, California","Alpine County, California","Alpine County, California","Alpine County, California","Alpine County, California","Amador County, California",...,"Ventura County, California","Ventura County, California","Yolo County, California","Yolo County, California","Yolo County, California","Yolo County, California","Yuba County, California","Yuba County, California","Yuba County, California","Yuba County, California"
2,Label,Estimate,Margin of Error,Percent,Percent Margin of Error,Estimate,Margin of Error,Percent,Percent Margin of Error,Estimate,...,Percent,Percent Margin of Error,Estimate,Estimate Margin of Error,Percent,Percent Margin of Error,Estimate,Estimate Margin of Error,Percent,Percent Margin of Error


In [278]:
hse_own_df = hse_ownership_df.copy()
hse_own_df = hse_own_df.T
hse_own_df.columns = hse_own_df.iloc[0]
hse_own_df = hse_own_df[1:].reset_index(drop=True)

In [279]:
hse_own_df["County"] = hse_own_df["County"].apply(
    lambda x: x.replace(" County, California", "").strip()
)
hse_own_df = hse_own_df.set_index(["County", "Year"])

In [280]:
hse_own_df.head(10)

0                               Label Total housing units  \
County  Year                                                
Alameda 2023                 Estimate             630,726   
        2023          Margin of Error                ±295   
        2023                  Percent             630,726   
        2023  Percent Margin of Error                 (X)   
Alpine  2023                 Estimate               1,587   
        2023          Margin of Error                 ±96   
        2023                  Percent               1,587   
        2023  Percent Margin of Error                 (X)   
Amador  2023                 Estimate              18,919   
        2023          Margin of Error                 ±47   

0            Occupied housing units Owner-occupied Renter-occupied  \
County  Year                                                         
Alameda 2023                593,117        320,712         272,405   
        2023                 ±1,676         ±2,874          ±2,581   
        2023                  94.0%          54.1%           45.9%   
        2023                   ±0.3           ±0.4            ±0.4   
Alpine  2023                    473            383              90   
        2023                   ±103            ±92             ±52   
        2023                  29.8%          81.0%           19.0%   
        2023                   ±5.8           ±9.7            ±9.7   
Amador  2023                 16,066         12,863           3,203   
        2023                   ±341           ±457            ±472   

0            Less than 15.0 percent 15.0 to 19.9 percent 20.0 to 24.9 percent  \
County  Year                                                                    
Alameda 2023                 32,899               34,476               33,678   
        2023                 ±1,393               ±1,609               ±1,269   
        2023                  12.6%                13.2%                12.9%   
        2023                   ±0.5                 ±0.6                 ±0.5   
Alpine  2023                     35                    2                   10   
        2023                    ±33                   ±4                  ±14   
        2023                  47.3%                 2.7%                13.5%   
        2023                  ±31.2                 ±4.9                ±17.0   
Amador  2023                    252                  472                  415   
        2023                   ±117                 ±227                 ±176   

0            25.0 to 29.9 percent 30.0 to 34.9 percent 35.0 percent or more  
County  Year                                                                 
Alameda 2023               29,869               23,770              106,282  
        2023               ±1,279               ±1,155               ±2,542  
        2023                11.4%                 9.1%                40.7%  
        2023                 ±0.5                 ±0.5                 ±0.8  
Alpine  2023                   22                    0                    5  
        2023                  ±25                  ±14                   ±6  
        2023                29.7%                 0.0%                 6.8%  
        2023                ±26.5                ±39.4                ±10.3  
Amador  2023                  310                  333                1,064  
        2023                 ±132                 ±202                 ±333

In [281]:
hse_own_df = hse_own_df.map(lambda x: x.replace(",", "").strip("±, %"))
hse_own_df["Total housing units"] = hse_own_df["Total housing units"].apply(
    lambda x: x.replace("(X)", "0")
)

In [282]:
hse_own_df[hse_own_df.columns[1:]] = hse_own_df[hse_own_df.columns[1:]].astype(
    "float64"
)

In [283]:
hse_own_df["home_ownership_rate"] = (
    hse_own_df["Owner-occupied"] / hse_own_df["Occupied housing units"]
)

In [284]:
hse_own_df

0                                Label  Total housing units  \
County  Year                                                  
Alameda 2023                  Estimate             630726.0   
        2023           Margin of Error                295.0   
        2023                   Percent             630726.0   
        2023   Percent Margin of Error                  0.0   
Alpine  2023                  Estimate               1587.0   
...                                ...                  ...   
Yolo    2010   Percent Margin of Error                  0.0   
Yuba    2010                  Estimate              27351.0   
        2010  Estimate Margin of Error                220.0   
        2010                   Percent              27351.0   
        2010   Percent Margin of Error                  0.0   

0             Occupied housing units  Owner-occupied  Renter-occupied  \
County  Year                                                            
Alameda 2023                593117.0        320712.0         272405.0   
        2023                  1676.0          2874.0           2581.0   
        2023                    94.0            54.1             45.9   
        2023                     0.3             0.4              0.4   
Alpine  2023                   473.0           383.0             90.0   
...                              ...             ...              ...   
Yolo    2010                     0.8             1.1              1.1   
Yuba    2010                 23750.0         14214.0           9536.0   
        2010                   494.0           525.0            540.0   
        2010                    86.8            59.8             40.2   
        2010                     1.7             2.0              2.0   

0             Less than 15.0 percent  15.0 to 19.9 percent  \
County  Year                                                 
Alameda 2023                 32899.0               34476.0   
        2023                  1393.0                1609.0   
        2023                    12.6                  13.2   
        2023                     0.5                   0.6   
Alpine  2023                    35.0                   2.0   
...                              ...                   ...   
Yolo    2010                     1.4                   1.2   
Yuba    2010                   603.0                1037.0   
        2010                   142.0                 290.0   
        2010                     7.2                  12.4   
        2010                     1.7                   3.2   

0             20.0 to 24.9 percent  25.0 to 29.9 percent  \
County  Year                                               
Alameda 2023               33678.0               29869.0   
        2023                1269.0                1279.0   
        2023                  12.9                  11.4   
        2023                   0.5                   0.5   
Alpine  2023                  10.0                  22.0   
...                            ...                   ...   
Yolo    2010                   1.5                   1.3   
Yuba    2010                1029.0                1092.0   
        2010                 243.0                 247.0   
        2010                  12.3                  13.0   
        2010                   2.6                   2.8   

0             30.0 to 34.9 percent  35.0 percent or more  home_ownership_rate  
County  Year                                                                   
Alameda 2023               23770.0              106282.0             0.540723  
        2023                1155.0                2542.0             1.714797  
        2023                   9.1                  40.7             0.575532  
        2023                   0.5                   0.8             1.333333  
Alpine  2023                   0.0                   5.0             0.809725  
...                            ...                   ...                  ...  
Yo

# Work on category: "Urban", "Suburban", "Rural"

In [285]:
# Define the criteria for Urban, Suburban, and Rural
conditions = [
    (crime_data_df["population_density"] > 1500),
    (crime_data_df["population_density"] <= 1500)
    & (crime_data_df["population_density"] >= 500),
    (crime_data_df["population_density"] < 500),
]

# The Categories: Index must much index of conditions
categories = ["Urban", "Suburban", "Rural"]

In [286]:
# Create the categorical variable 'area_type'
crime_data_df["county_category"] = np.select(conditions, categories, default="unknown")

In [287]:
crime_data_df

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   
        1988        10963           159          722         4863   
        1989        10563           172          670         4879   
...                   ...           ...          ...          ...   
Yuba    2019          326             7           41           61   
        2020          404             2           37           57   
        2021          354             6           35           70   
        2022          279             5           33           40   
        2023          334             9           24           31   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   
        1988            5219            5708              100             498   
        1989            4842            5250               98             453   
...                      ...             ...              ...             ...   
Yuba    2019             217             134                8               6   
        2020             308             161                2               5   
        2021             243             149                7               9   
        2022             201             121                4               7   
        2023             270             122                7               8   

              RobberyClr_sum  AggAssaultClr_sum  ...  Population  Area_sq_mi  \
County  Year                                     ...                           
Alameda 1985            1517               3376  ...     1185500       738.0   
        1986            1545               3379  ...     1206900       738.0   
        1987            1569               4132  ...     1220600       738.0   
        1988            1545               3565  ...     1242300       738.0   
        1989            1496               3203  ...     1261200       738.0   
...                      ...                ...  ...         ...         ...   
Yuba    2019              13                107  ...       79619       630.0   
        2020              26                128  ...       81178       630.0   
        2021              21                112  ...       82091       630.0   
        2022              18                 92  ...       82563       630.0   
        2023              14                 93  ...       83405       630.0   

              crime_rate  clearance_rate  population_density  \
County  Year                                                   
Alameda 1985    0.009809        0.004580         1606.368564   
        1986    0.010353        0.004615         1635.365854   
        1987    0.009588        0.005164         1653.929539   
        1988    0.008825        0.004595         1683.333333   
        1989    0.008375        0.004163         1708.943089   
...                  ...             ...                 ...   
Yuba    2019    0.004095        0.001683          126.379365   
        2020    0.004977        0.001983          128.853968   
        2021    0.004312        0.001815          130.303175   
        2022    0.003379        0.001466          131.052381   
        2023    0.004005        0.001463          132.388889   

              unemployment_rate  median_hse_income      CPI  adjusted_income  \
County  Year                                                                   
Alameda 1985                

# Work on Age folder to get the median age.

Conbine the Excels sheets

In [ ]:
# Define the directory with the excel files

age_dir = "../data/Age"

age_dfs = {} # Dictionary to store dataframes

# Loop over a list of files in the directory: Age
for filename in os.listdir(age_dir):
    # print(filename)
    file_path = os.path.join(age_dir, filename)
    raw_data = pd.read_excel(file_path, sheet_name="Data", header=[0, 1])
    tranpose = raw_data.T
    tranpose.columns = tranpose.iloc[0]
    tranpose = tranpose[1:]
    tranpose = tranpose.reset_index()
    tranpose = tranpose.rename(columns={"level_0": "County"})
    tranpose["County"] = tranpose["County"].apply(
        lambda x: x.replace(" County, California", "").strip()
    )
    year = filename.strip("Age_, .xlsx")
    tranpose["Year"] = year
    tranpose.set_index(["County", "Year"], inplace=True)
    # tranpose = tranpose.drop['SUMMARY INDICATORS','PERCENT ALLOCATED','Sex', 'Age']
    age_dfs[year] = tranpose

### Combine all the dataframes into one dataframe 

In [289]:
combined_age_dfs = pd.concat(age_dfs.values(), axis=0)
combined_age_dfs.head()

('Unnamed: 0_level_0', 'Unnamed: 0_level_1')  level_1            Label  \
County  Year                                                             
Alameda 2010                                    Total         Estimate   
        2010                                  Total.1  Margin of Error   
        2010                                     Male         Estimate   
        2010                                   Male.1  Margin of Error   
        2010                                   Female         Estimate   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Total population  AGE  \
County  Year                                                         
Alameda 2010                                        1,477,980  NaN   
        2010                                            *****  NaN   
        2010                                          725,077  NaN   
        2010                                             ±181  NaN   
        2010                                          752,903  NaN   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Under 5 years 5 to 9 years  \
County  Year                                                              
Alameda 2010                                          6.6%         6.3%   
        2010                                         *****         ±0.1   
        2010                                          7.0%         6.6%   
        2010                                          ±0.1         ±0.1   
        2010                                          6.3%         6.0%   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') 10 to 14 years 15 to 19 years  \
County  Year                                                                 
Alameda 2010                                           6.1%           6.7%   
        2010                                           ±0.1           ±0.1   
        2010                                           6.3%           7.0%   
        2010                                           ±0.1           ±0.1   
        2010                                           5.9%           6.4%   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') 20 to 24 years 25 to 29 years  \
County  Year                                                                 
Alameda 2010                                           7.0%           7.7%   
        2010                                           ±0.1          *****   
        2010                                           7.2%           7.8%   
        2010                                           ±0.1           ±0.1   
        2010                                           6.7%           7.7%   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1')  ... Age dependency ratio  \
County  Year                                  ...                        
Alameda 2010                                  ...                 50.9   
        2010                                  ...                *****   
        2010                                  ...                  (X)   
        2010                                  ...                  (X)   
        2010                                  ...                  (X)   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Old-Age dependency ratio  \
County  Year                                                            
Alameda 2010                                                     16.3   
        2010                                                    *****   
        2010                                                      (X)   
        2010                                                      (X)   
        2010                                                      (X)   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1') Child dependency ratio  \
County  Year                                                          
Alameda 2010                                                   34.6   
        2010                                                  *****   
        2010                                           

In [290]:
crime_data_df["median_age"] = combined_age_dfs[combined_age_dfs["level_1"] == "Total"][
    "Median age (years)"
]

In [291]:
crime_data_df.head(5)

Violent_sum  Homicide_sum  ForRape_sum  Robbery_sum  \
County  Year                                                        
Alameda 1985        11628           143          791         5427   
        1986        12495           174          820         5971   
        1987        11703           147          770         5019   
        1988        10963           159          722         4863   
        1989        10563           172          670         4879   

              AggAssault_sum  ViolentClr_sum  HomicideClr_sum  ForRapeClr_sum  \
County  Year                                                                    
Alameda 1985            5267            5429               91             445   
        1986            5530            5570              114             532   
        1987            5767            6303               91             511   
        1988            5219            5708              100             498   
        1989            4842            5250               98             453   

              RobberyClr_sum  AggAssaultClr_sum  ...  Area_sq_mi  crime_rate  \
County  Year                                     ...                           
Alameda 1985            1517               3376  ...       738.0    0.009809   
        1986            1545               3379  ...       738.0    0.010353   
        1987            1569               4132  ...       738.0    0.009588   
        1988            1545               3565  ...       738.0    0.008825   
        1989            1496               3203  ...       738.0    0.008375   

              clearance_rate  population_density  unemployment_rate  \
County  Year                                                          
Alameda 1985        0.004580         1606.368564                NaN   
        1986        0.004615         1635.365854                NaN   
        1987        0.005164         1653.929539                NaN   
        1988        0.004595         1683.333333                NaN   
        1989        0.004163         1708.943089                NaN   

              median_hse_income  CPI  adjusted_income  county_category  \
County  Year                                                             
Alameda 1985                NaN  NaN              NaN            Urban   
        1986                NaN  NaN              NaN            Urban   
        1987                NaN  NaN              NaN            Urban   
        1988                NaN  NaN              NaN            Urban   
        1989                NaN  NaN              NaN            Urban   

              median_age  
County  Year              
Alameda 1985         NaN  
        1986         NaN  
        1987         NaN  
        1988         NaN  
        1989         NaN  

[5 rows x 29 columns]

# Work on School Enrollment

In [299]:
from pathlib import Path

In [ ]:

sch_enrol_dir = "../data/School_Enrollment"

sch_enrol_dfs = {} # A dictionary to store the Dataframes
raw_data = pd.DataFrame()

# Loop over a list of files in the directory: School_Enrollment
for filename in os.listdir(sch_enrol_dir):
    # print(filename)
    file_path = os.path.join(sch_enrol_dir, filename)
    # Convert the string to a Path object
    file_path = Path(file_path)
    if file_path.suffix == '.xlsx':
        raw_data = pd.read_excel(file_path, sheet_name="Data", header=[0,1,2,3]) 
    if file_path.suffix == '.csv':
        raw_data = pd.read_csv(file_path, header=[0,1,2,3]) 
    year = filename.replace('School_enrollment_', '').replace('.xlsx', '').replace('.csv', '')
    trans = raw_data.T
    trans["Year"] = year
    sch_enrol_dfs[year] = trans

In [372]:
sch_enrol_dfs['2010']

0  \
Unnamed: 0_level_0         Unnamed: 0_level_1              Unnamed: 0_level_2   Label            Population 3 years and over enrolled in school   
Alameda County, California Total                           Unnamed: 1_level_2   Estimate                                                407,491   
                                                           Unnamed: 2_level_2   Margin of Error                                          ±3,036   
                           Percent of enrolled population  In public school     Estimate                                                  84.4%   
                                                                                Margin of Error                                            ±0.5   
...                                                                                                                                         ...   
Yuba County, California    Total                           Unnamed: 344_level_2 Margin of Error                                            ±484   
                           Percent of enrolled population  In public school     Estimate                                                  91.9%   
                                                                                Margin of Error                                            ±1.7   
                                                           In private school    Estimate                                                   8.1%   
                                                                                Margin of Error                                            ±1.7   

                                                                                                                         1  \
Unnamed: 0_level_0         Unnamed: 0_level_1              Unnamed: 0_level_2   Label            Nursery school, preschool   
Alameda County, California Total                           Unnamed: 1_level_2   Estimate                            26,241   
                                                           Unnamed: 2_level_2   Margin of Error                       ±950   
                           Percent of enrolled population  In public school     Estimate                             44.0%   
                                                                                Margin of Error                       ±2.5   
...                                                                                                                    ...   
Yuba County, California    Total                           Unnamed: 344_level_2 Margin of Error                       ±214   
                           Percent of enrolled population  In public school     Estimate                             82.0%   
                                                                                Margin of Error                       ±7.7   
                                                           In private school    Estimate                             18.0%   
                                                                                Margin of Error                       ±7.7   

                                                                                                                          2  \
Unnamed: 0_level_0         Unnamed: 0_level_1              Unnamed: 0_level_2   Label            Kindergarten to 12th grade   
Alameda County, California Total                           Unnamed: 1_level_2   Estimate                            244,741   
                                                           Unnamed: 2_level_2   Margin of Error                      ±1,355   
                           Percent of enrolled population  In public school     Estimate                              89.0%   
                                                                                Margin of Error                        ±0.5   
...                                                                                                   

### Combine all the dataframes into one dataframe 

In [382]:
combine_sch_enrol = pd.concat(sch_enrol_dfs.values(), axis=0)
combine_sch_enrol.columns= combine_sch_enrol.iloc[0]
combine_sch_enrol=combine_sch_enrol[1:]
combine_sch_enrol = combine_sch_enrol.reset_index()
combine_sch_enrol=combine_sch_enrol.rename(columns={'2010':'Year', 'level_0':'County', 'level_3':'Label'}) 
combine_sch_enrol["County"] = combine_sch_enrol["County"].apply(
    lambda x: x.replace(" County, California", "").strip()
)
combine_sch_enrol = combine_sch_enrol.set_index(['County', 'Year'])


In [384]:
combine_sch_enrol.head()

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label')                          level_1  \
County  Year                                                                                                   
Alameda 2010                                                                                           Total   
        2010                                                                                           Total   
        2010                                                                 Percent of enrolled population    
        2010                                                                 Percent of enrolled population    
        2010                                                                 Percent of enrolled population    

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label')             level_2  \
County  Year                                                                                      
Alameda 2010                                                                 Unnamed: 1_level_2   
        2010                                                                 Unnamed: 2_level_2   
        2010                                                                   In public school   
        2010                                                                   In public school   
        2010                                                                  In private school   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label')            Label  \
County  Year                                                                                   
Alameda 2010                                                                        Estimate   
        2010                                                                 Margin of Error   
        2010                                                                        Estimate   
        2010                                                                 Margin of Error   
        2010                                                                        Estimate   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label') Population 3 years and over enrolled in school  \
County  Year                                                                                                                 
Alameda 2010                                                                                                       407,491   
        2010                                                                                                        ±3,036   
        2010                                                                                                         84.4%   
        2010                                                                                                          ±0.5   
        2010                                                                                                         15.6%   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label') Nursery school, preschool  \
County  Year                                                                                            
Alameda 2010                                                                                   26,241   
        2010                                                                                     ±950   
        2010                                                                                    44.0%   
        2010                                                                                     ±2.5   
        2010                                                                                    56.0%   

('Unnamed: 0_level_0', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2', 'Label') Kindergarten to 12th grade  \
County  Year                                                                                             
Alameda 2010     

# Work on Expenditure

In [393]:
expenditure_df = pd.read_excel("../data/Expenditure_2003_to_2023.xlsx")

In [394]:
expenditure_df.head(4)

,Entity Name,Entity ID,Fiscal Year,Police Protection_Total Governmental Funds_,Total Education_Total Governmental Funds,Total Public Assistance_Total Governmental Funds,Mental Health_Total Governmental Funds_Health,Drug and Alcohol Abuse Services_Total Governmental Funds_Health,Total Health_Total Governmental Funds,Total Judicial_Total Governmental Funds,Total Detention and Correction_Total Governmental Funds
0,Alameda,1,2023,136000000,39064698.0,1044985620,503588373.0,59132457.0,1100494826,253504410,433901696
1,Alpine,2,2023,3803427,684938.0,3104497,1997691.0,267679.0,3897922,573927,704258
2,Amador,3,2023,13793355,1087355.0,16887389,9895140.0,936709.0,14857378,7175470,9139719
3,Butte,4,2023,23968804,4206603.0,156340270,68213421.0,6190888.0,98478974,33032555,52562192


In [395]:
expenditure_df.columns

Index(['Entity Name', 'Entity ID', 'Fiscal Year',
       'Police Protection_Total Governmental Funds_',
       'Total Education_Total Governmental Funds',
       'Total Public Assistance_Total Governmental Funds',
       'Mental Health_Total Governmental Funds_Health',
       'Drug and Alcohol Abuse Services_Total Governmental Funds_Health',
       'Total Health_Total Governmental Funds',
       'Total Judicial_Total Governmental Funds',
       'Total Detention and Correction_Total Governmental Funds'],
      dtype='object')

In [396]:
expenditure_df = expenditure_df.rename(columns={'Entity Name': 'County', 'Fiscal Year':'Year'})

In [ ]:
expenditure_df = expenditure_df.set_index(['County', 'Year'])

,,Entity ID,Police Protection_Total Governmental Funds_,Total Education_Total Governmental Funds,Total Public Assistance_Total Governmental Funds,Mental Health_Total Governmental Funds_Health,Drug and Alcohol Abuse Services_Total Governmental Funds_Health,Total Health_Total Governmental Funds,Total Judicial_Total Governmental Funds,Total Detention and Correction_Total Governmental Funds
County,Year,,,,,,,,,
Alameda,2023,1,136000000,39064698.0,1044985620,503588373.0,59132457.0,1100494826,253504410,433901696
Alpine,2023,2,3803427,684938.0,3104497,1997691.0,267679.0,3897922,573927,704258
Amador,2023,3,13793355,1087355.0,16887389,9895140.0,936709.0,14857378,7175470,9139719
Butte,2023,4,23968804,4206603.0,156340270,68213421.0,6190888.0,98478974,33032555,52562192
Calaveras,2023,5,13471436,1054402.0,26934694,13070812.0,1200171.0,21193409,5183073,9579066


In [400]:
expenditure_df.head(50)

,,Entity ID,Police Protection_Total Governmental Funds_,Total Education_Total Governmental Funds,Total Public Assistance_Total Governmental Funds,Mental Health_Total Governmental Funds_Health,Drug and Alcohol Abuse Services_Total Governmental Funds_Health,Total Health_Total Governmental Funds,Total Judicial_Total Governmental Funds,Total Detention and Correction_Total Governmental Funds
County,Year,,,,,,,,,
Alameda,2023,1,136000000,39064698.0,1044985620,5.035884e+08,59132457.0,1100494826,253504410,433901696
Alpine,2023,2,3803427,684938.0,3104497,1.997691e+06,267679.0,3897922,573927,704258
Amador,2023,3,13793355,1087355.0,16887389,9.895140e+06,936709.0,14857378,7175470,9139719
Butte,2023,4,23968804,4206603.0,156340270,6.821342e+07,6190888.0,98478974,33032555,52562192
Calaveras,2023,5,13471436,1054402.0,26934694,1.307081e+07,1200171.0,21193409,5183073,9579066
Colusa,2023,6,7829504,1193397.0,13693497,1.215122e+07,752539.0,20288594,3986775,8736098
Contra Costa,2023,7,232000000,38418100.0,616905208,2.057681e+08,18283736.0,368830012,119753157,188893018
Del Norte,2023,8,4780030,90797.0,30288024,9.971184e+06,1127830.0,14999338,4148599,8540026
El Dorado,2023,9,53438607,4172017.0,83673581,3.091946e+07,4936155.0,72701186,29761470,41183923
